In [1]:
import json

In [19]:
with open('../2/preprocess/data.json', mode='r') as f:
    data = json.load(f)
levels = [j.strip() for i in data for j in i['levels']]
foods = [i['name'].strip() for i in data]
ingredients_foods = {j['name'].strip():[] for i in data for j in i['ingredients']}
for i in data:
    for j in i['ingredients']:
        ingredients_foods[j['name'].strip()].append(i['name'])
with open('stopwords.txt', 'r') as f:
    stopwords = {i.strip() for i in f}

<div dir=rtl style="font-size: 30px; font-family: XB Niloofar;">
پیش پردازش و حذف stopwordها
</div>

<div dir=rtl style="font-size: 18px; font-family: XB Niloofar;">
تمامی اعراب‌ها و علائم نگارش حذف شده‌اند. stopwordها نیز که فرکانس زیادی دارند و می‌توانند ما را دچار اشتباه کنند حذف شده‌اند. در نهایت راس‌های گراف ساخته‌شده، مراحل پخت و پز هستند و دو راس را به هم وصل می‌کنیم هرگاه که اشتراک کلماتشان از مقدار threshold بیشتر باشد.
</div>

In [20]:
def normalizer(l):
    normalized_l = []
    for i in l:
        n = []
        for c in i:
            if c in {
                'ْ',
                'ٌ',
                'ٍ',
                'ً',
                'ُ',
                'ِ',
                'َ',
                'ّ',
                ']',
                '[',
                ')',
                '(',
            }:
                pass
            elif c == 'آ' or c == 'أ' or c == 'إ':
                n.append('ا')
            elif c == '.' or c == '،':
                n.append(' ')
            else:
                n.append(c)
        normalized_l.append(''.join(n))
    return normalized_l

normalized_levels = normalizer(levels)

In [21]:
import networkx as nx

In [22]:
G = nx.Graph()

In [23]:
G.add_nodes_from(normalized_levels)

In [7]:
threshold = 3
without_stopwords = [set(i.split()) - stopwords for i in normalized_levels]
for i in range(len(normalized_levels)):
    s1 = without_stopwords[i]
    for j in range(i + 1, len(normalized_levels)):
        s2 = without_stopwords[j]
        if len(s1.intersection(s2)) >= threshold:
            G.add_edge(normalized_levels[i], normalized_levels[j])

<div dir=rtl style="font-size: 30px; font-family: XB Niloofar;">
الگوریتم PageRank
</div>

In [8]:
pr = nx.pagerank(G)

In [9]:
sorted([(i, j) for i, j in pr.items()], key=lambda x: x[1], reverse=True)[:20]

[('برای تهیه سوپ مرغ مجلسی ابتدا یکی از پیازها را به صورت ۴ قاچ خرد می کنیم  سپس به همراه مقداری نمک  زردچوبه و گوشت مرغ داخل یک قابلمه مناسب می ریزیم  در ادامه ۲ لیوان اب اضافه کرده و قابلمه را روی حرارت قرار می دهیم ',
  0.0004896474742519734),
 ('در این مرحله پیاز سرخ شده را به پوره سیب زمینی اضافه می کنیم  سپس هم می زنیم تا مواد ترکیب و یکدست شوند  در ادامه خمیر یوفکا را روی سطح کار قرار می دهیم  سپس شیر را به همراه مقداری روغن درون یک کاسه مناسب می ریزیم ',
  0.0004884669522184022),
 ('حالا پیاز ها را به صورت نگینی ریز خرد می کنیم و یک قابلمه مناسب روی حرارت ملایم قرار می دهیم  در ادامه مقداری روغن مایع به تابه اضافه می کنیم  سپس تکه های گوشت را به همراه فلفل سیاه و زردچوبه به قابلمه می افزاییم ',
  0.00048446009947592384),
 ('در این مرحله گردو را درون ابکش می ریزیم و اجازه می دهیم اب اضافی ان خارج شود  در ادامه یک تابه مناسب روی حرارت ملایم قرار می دهیم  سپس مقداری روغن مایع و کره درون ان می ریزیم و پیاز را اضافه می کنیم ',
  0.0004825052573600331),
 ('حالا داخل یک کاسه دیگر مقدا

<div dir=rtl style="font-size: 30px; font-family: XB Niloofar;">
    الگوریتم HITS
</div>

In [10]:
hubs, authorities = nx.hits(G)

/home/hamid/.local/lib/python3.10/site-packages/networkx/algorithms/link_analysis/hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


In [11]:
hubs

{'برای تهیه همبرگر خوشمزه و مخصوص به روش خانگی در ابتدای کار  گوشت چرخ کرده را درون یک کاسه مناسب می ریزیم  در ادامه پیاز رنده شده را به ان اضافه می کنیم  حالا مقداری نمک و پاپریکا و پودر سیر اضافه می کنیم ': 0.00032748870485620424,
 'در این مرحله سس خردل را نیز به کاسه حاوی مواد اضافه می کنیم  سپس به مدت ۱۰ دقیقه مایه همبرگر را به خوبی با دست ورز می دهیم تا کاملا یکدست و منسجم شود  حالا روی ظرف حاوی مایه همبرگر را با سلفون می پوشانیم ': 0.0002638662644020824,
 'حالا مایه همبرگر را به مدت حداقل یک تا سه ساعت درون یخچال قرار می دهیم تا استراحت کند و به خوبی مزه دار شود  پس از اینکه مایه همبرگر به خوبی استارحت کرد  انرا از یخچال خارج می کنیم  سپس به سطح کار منتقل می کنیم ': 0.00010966895767770448,
 'در این مرحله مایه همبرگر را به ۴ قسمت مساوی تقسیم می کنیم  هر قسمت را به صورت قلقلی درون دست گرد می کنیم و کنار می گذاریم  حالا قلقلی های گوشت را درون قالب همبرگر زن قرار داده و به انها شکل می دهیم ': 0.0002525417231082886,
 'در صورتی که قالب همبرگر زن در دسترس نداشتیم  می توانیم با بشقاب این

In [12]:
authorities

{'برای تهیه همبرگر خوشمزه و مخصوص به روش خانگی در ابتدای کار  گوشت چرخ کرده را درون یک کاسه مناسب می ریزیم  در ادامه پیاز رنده شده را به ان اضافه می کنیم  حالا مقداری نمک و پاپریکا و پودر سیر اضافه می کنیم ': 0.00032748870485620424,
 'در این مرحله سس خردل را نیز به کاسه حاوی مواد اضافه می کنیم  سپس به مدت ۱۰ دقیقه مایه همبرگر را به خوبی با دست ورز می دهیم تا کاملا یکدست و منسجم شود  حالا روی ظرف حاوی مایه همبرگر را با سلفون می پوشانیم ': 0.00026386626440208245,
 'حالا مایه همبرگر را به مدت حداقل یک تا سه ساعت درون یخچال قرار می دهیم تا استراحت کند و به خوبی مزه دار شود  پس از اینکه مایه همبرگر به خوبی استارحت کرد  انرا از یخچال خارج می کنیم  سپس به سطح کار منتقل می کنیم ': 0.00010966895767770451,
 'در این مرحله مایه همبرگر را به ۴ قسمت مساوی تقسیم می کنیم  هر قسمت را به صورت قلقلی درون دست گرد می کنیم و کنار می گذاریم  حالا قلقلی های گوشت را درون قالب همبرگر زن قرار داده و به انها شکل می دهیم ': 0.0002525417231082886,
 'در صورتی که قالب همبرگر زن در دسترس نداشتیم  می توانیم با بشقاب ای

<div dir=rtl style="font-size: 30px; font-family: XB Niloofar;">
گراف موادغذایی-غذا
</div>

<div dir=rtl style="font-size: 18px; font-family: XB Niloofar;">
این گراف یک گراف دو بخشی است که یک بخش آن مواد غذایی و بخش دیگر آن غذاها هستند. بین یک غذا و ماده غذایی یال وجود دارد اگر که آن ماده غذایی در غذا به کار رفته باشد.
</div>

In [13]:
G = nx.Graph()
G.add_nodes_from(foods)
G.add_nodes_from(ingredients_foods.keys())

In [14]:
for i, j in ingredients_foods.items():
    for k in j:
        G.add_edge(i, k)

In [15]:
pr = nx.pagerank(G)
sorted([(i, j) for i, j in pr.items()], key=lambda x: x[1], reverse=True)[:20]

[('پیاز', 0.026124946742432376),
 ('نمک و فلفل سیاه', 0.017735209016758),
 ('سیر', 0.013128213401983776),
 ('رب گوجه فرنگی', 0.010884610328635809),
 ('تخم مرغ', 0.009593870585001244),
 ('سیب زمینی', 0.009242020297984021),
 ('زعفران دم کرده', 0.00882477540473704),
 ('کره', 0.008063183262550878),
 ('روغن مایع', 0.006861034273705129),
 ('زردچوبه و روغن', 0.0067872038939213045),
 ('نمک', 0.006621291351639152),
 ('گوجه فرنگی', 0.006340327713543388),
 ('شکر', 0.005769387901419154),
 ('برنج', 0.005527071737802058),
 ('گوشت چرخ کرده', 0.0053838229205723925),
 ('سینه مرغ', 0.004741441129611775),
 ('روغن زیتون', 0.00460215110161053),
 ('آب', 0.004502770350967628),
 ('آرد', 0.00441122205185558),
 ('شیر', 0.004205187948461674)]

In [16]:
G = nx.DiGraph()
G.add_nodes_from(foods)
G.add_nodes_from(ingredients_foods.keys())
for i, j in ingredients_foods.items():
    for k in j:
        G.add_edge(k, i)
hubs, authorities = nx.hits(G)

In [17]:
sorted([(i, j) for i, j in hubs.items()], key=lambda x: x[1], reverse=True)[:20]

[('لازانیا رولی', 0.006766271742628389),
 ('تاس کباب ساده', 0.006502815759971285),
 ('زرشک پلو با مرغ ساده', 0.0062362845936886225),
 ('رولت مرغ ساده', 0.006128911646600745),
 ('لازانیا ساده', 0.005820210086872028),
 ('املت گوجه فرنگی ساده', 0.0057465682639230905),
 ('کتلت آبدار ساده', 0.0057128542509404315),
 ('سوپ مرغ مجلسی', 0.005668480166760081),
 ('کتلت سویا ساده', 0.005608504241813659),
 ('ته چین کباب تابه ای', 0.005586208590087788),
 ('ساندویچ مرغ ساده', 0.0055364791978684495),
 ('یتیمچه تهرانی', 0.005471158785530313),
 ('پیراشکی سیب زمینی', 0.005403840119139728),
 ('چلو گوشت ساده', 0.005345940433329728),
 ('خورش نخود فرنگی', 0.005331657623770334),
 ('خورش مرغ ساده', 0.005323192032743277),
 ('کتلت مرغ ساده', 0.005322293983136505),
 ('شنیسل مرغ ساده', 0.005298804921443825),
 ('خورش انار دون', 0.005272536778311726),
 ('پیراشکی گوشت ساده', 0.005252322394279761)]

In [18]:
sorted([(i, j) for i, j in authorities.items()], key=lambda x: x[1], reverse=True)[:20]

[('پیاز', 0.08047246176426727),
 ('نمک و فلفل سیاه', 0.057118413293325676),
 ('سیر', 0.03702287127544396),
 ('رب گوجه فرنگی', 0.034393671830276244),
 ('زردچوبه و روغن', 0.02806009037434736),
 ('سیب زمینی', 0.025729279126280428),
 ('تخم مرغ', 0.025535909095846576),
 ('زعفران دم کرده', 0.02504531995803394),
 ('کره', 0.020700119388254213),
 ('گوجه فرنگی', 0.019112804591007594),
 ('گوشت چرخ کرده', 0.0167918076273954),
 ('برنج', 0.01555456589465318),
 ('سینه مرغ', 0.013064407099767451),
 ('فلفل دلمه ای', 0.012719375981269477),
 ('روغن مایع', 0.011747974139602092),
 ('هویج', 0.011573193478351507),
 ('آرد', 0.011236306460091801),
 ('نمک', 0.011210933069354959),
 ('قارچ', 0.011170832988053696),
 ('روغن زیتون', 0.01010097234971819)]